In [126]:
import re
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [141]:
file = open('C:\\Users\\Margarita\\Desktop\\test_margaryta.txt','r',encoding='utf-8')
text = file.readlines()
#text

In [142]:
# Переходим от формата str к dict
lst = []

for string in text:
    a=eval(string[:-2])
    lst.append(a)

In [143]:
# убираем из текста \\xa0 и теги html

for topic in range(len(lst)):
    for line in range(len(lst[topic]['article'])):
        lst[topic]['article'][line] = lst[topic]['article'][line].replace(u'\xa0', u' ')
        lst[topic]['article'][line] = re.sub('<[^<]+?>', '', lst[topic]['article'][line])

In [144]:
lst[0]['article']

['«АвтоВАЗ» в апреле продал в общей сложности 10055 автомобилей, сообщила пресс-служба автоконцерна. Самой популярной была Granta с результатом 3058 автомобилей, на втором месте Vesta — 2913 проданных автомобилей, на третьем — Largus — 1286 автомобилей.',
 'По сравнению с апрелем 2019 года, сообщает ТАСС, продажи «АвтоВАЗа» упали более чем в 3 раза. В прошлом году автоконцерн продал в апреле 32,3 тысячи автомобилей.']

In [145]:
# обработка текста

In [146]:
# переносим все статьи('article') в отдельный список, для дальнейшей обработки
data = []

for topic in range(len(lst)):
    text = str()
    for sentence in lst[topic]['article']:
        text += sentence + ' '
    data.append(text)

In [147]:
# приводим текст к нижнему регистру
data = [line.lower() for line in data]

# удаляем знаки пунктуации
data = [re.sub(r'[^\w\s]',' ',line,flags=re.UNICODE) for line in data]

# удаляем знаки цифры
data = [re.sub(r'[\d]',' ',line,flags=re.UNICODE) for line in data]

# токенизация текстов
tokenized_data = [nltk.word_tokenize(line) for line in data]

print(tokenized_data[0])

['автоваз', 'в', 'апреле', 'продал', 'в', 'общей', 'сложности', 'автомобилей', 'сообщила', 'пресс', 'служба', 'автоконцерна', 'самой', 'популярной', 'была', 'granta', 'с', 'результатом', 'автомобилей', 'на', 'втором', 'месте', 'vesta', 'проданных', 'автомобилей', 'на', 'третьем', 'largus', 'автомобилей', 'по', 'сравнению', 'с', 'апрелем', 'года', 'сообщает', 'тасс', 'продажи', 'автоваза', 'упали', 'более', 'чем', 'в', 'раза', 'в', 'прошлом', 'году', 'автоконцерн', 'продал', 'в', 'апреле', 'тысячи', 'автомобилей']


In [148]:
# уберем "стоп-слова", т.е. наиболее часто используемые слова не несущие смысловой нагрузки ('и','чуть' и пр.) 

stop_words = stopwords.words("russian")

def lineWithoutStopWords(line):
    return [word for word in line if word not in stop_words]

withoutStopWords = [lineWithoutStopWords(line) for line in tokenized_data]
print(withoutStopWords[0])

['автоваз', 'апреле', 'продал', 'общей', 'сложности', 'автомобилей', 'сообщила', 'пресс', 'служба', 'автоконцерна', 'самой', 'популярной', 'granta', 'результатом', 'автомобилей', 'втором', 'месте', 'vesta', 'проданных', 'автомобилей', 'третьем', 'largus', 'автомобилей', 'сравнению', 'апрелем', 'года', 'сообщает', 'тасс', 'продажи', 'автоваза', 'упали', 'раза', 'прошлом', 'году', 'автоконцерн', 'продал', 'апреле', 'тысячи', 'автомобилей']


In [135]:
# Лемматизация 

mystem = Mystem()
lemma_lst =[]

for topic in range(len(withoutStopWords)):
    text = str()
    for word in withoutStopWords[topic]:
        text += mystem.lemmatize(word)[0] + ' '
    lemma_lst.append(text)
    print(topic)

0
1
2
3
4
5
6
7


In [149]:
print(lemma_lst[0])

автоваз апрель продавать общий сложность автомобиль сообщать пресс служба автоконцерн самый популярный granta результат автомобиль второй место vesta продавать автомобиль третий largus автомобиль сравнение апрель год сообщать тасс продажа автоваз упасть раз прошлое год автоконцерн продавать апрель тысяча автомобиль 


In [137]:
# подготовим данные для TfidfVectorizer, чтобы вычислить TF-IDF

to_Vectorize = []

for topic in lemma_lst:
    tmp=[]
    tmp.append(topic)
    to_Vectorize.append(tmp)

In [138]:
tfidf_vectorizer = TfidfVectorizer()

def tfidf_values(vect):
    values=tfidf_vectorizer.fit_transform(vect)
    feature_names = tfidf_vectorizer.get_feature_names()
    return pd.DataFrame(values.toarray(), columns = feature_names)

In [139]:
# TF-IDF
Vectors_lst=[]
for i in range(len(to_Vectorize)):
    Vectors_lst.append(tfidf_values(to_Vectorize[i]))

In [140]:
Vectors_lst[0]

,granta,largus,vesta,автоваз,автоконцерн,автомобиль,апрель,второй,год,место,...,результат,самый,сложность,служба,сообщать,сравнение,тасс,третий,тысяча,упасть
0,0.112509,0.112509,0.112509,0.225018,0.225018,0.562544,0.337526,0.112509,0.225018,0.112509,...,0.112509,0.112509,0.112509,0.112509,0.225018,0.112509,0.112509,0.112509,0.112509,0.112509
